In [40]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException,ElementNotInteractableException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait


In [41]:
#Parámetros
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}

options=webdriver.EdgeOptions()
options.add_argument("-inprivate")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-cache")
options.add_argument("--disable-cookies")

In [78]:
def data_walmart(zip_code,busqueda):
    busqueda=busqueda.replace(" ","+")

    #Cambia ubicación para obtener precios
    url="https://super.walmart.com.mx/"
    driver=webdriver.Edge(options=options)
    driver.get(url)
    time.sleep(5)
    try:
        elemento=driver.find_element(By.NAME,"select-store-button")
    except NoSuchElementException:
        elemento = driver.find_element(By.CSS_SELECTOR, 'img[src="https://i5.walmartimages.com/dfw/4ff9c6c9-14f0/k2-_8ce34043-6050-4f39-afb2-3e76a554f8a3.v1.png"]')
    elemento.click()
    #Ingresa código postal
    time.sleep(4)
    cp=driver.find_element(By.CSS_SELECTOR,"input[data-automation-id='store-zip-code']")
    cp.clear()
    cp.send_keys(zip_code)
    time.sleep(4)
    #Obtener datos de la tienda
    tienda = driver.find_element(By.CSS_SELECTOR, 'span.b.f5.lh-copy.dark-gray').text
    address = driver.find_element(By.CSS_SELECTOR, 'span.f6[aria-label]').get_attribute('aria-label')
    radio_buttons=driver.find_element(By.NAME,'pickup-store')
    #Aceptar elección
    elegir=driver.find_element(By.CSS_SELECTOR,"button[data-automation-id='save-label']")
    elegir.click()
    time.sleep(4)

    ###buscar producto
    url_busqueda="https://www.walmart.com.mx/productos?Ntt="+busqueda
    driver.get(url_busqueda)
    time.sleep(4)
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    results = soup.find_all('div', {'data-stack-index': '0'})
    span_tags = []
    product_titles = []

    # Loop para obtener precios y productos
    for result in results:
        span_tags.extend(result.select('div[data-automation-id="product-price"] span'))
        product_titles.extend([title.text for title in result.select('span[data-automation-id="product-title"]')])

    precios = []

    for span in span_tags:
        text = span.text
        if 'precio actual' in text:
            precios.append(text.split("precio actual")[1].strip())
    #Fecha y hora de la consulta
    fecha=time.strftime("%d/%m/%y")
    hora=time.strftime("%H:%M:%S")
    fecha_hora=hora+" "+fecha
    df=pd.DataFrame({'producto':product_titles,'precio':precios,'tienda':tienda,'direccion':address,'busqueda':busqueda,'fecha_consulta':fecha_hora})
    #Limpiar precios
    #Dejar solo números utilizando expresiones regulares
    df['precio'] = df['precio'].str.replace(r'[^0-9.]', '', regex=True)
    #Eliminar comas
    df['precio']=df['precio'].str.replace(',','',regex=True)
    #Convertir a numérico
    df['precio']=pd.to_numeric(df['precio'])
    df['producto']=df.producto.str.lower()
    return df
    


In [83]:
data_walmart("03630","leche")

,producto,precio,tienda,direccion,busqueda,fecha_consulta
0,leche lala 100 sin lactosa proteína light 1 l,37.0,Tlalpan,"Calzada De Tlalpan No. 1037, Colonia Américas ...",leche,13:43:23 22/08/23
1,leche santa clara entera caja con 6 pzas de 1 ...,150.0,Tlalpan,"Calzada De Tlalpan No. 1037, Colonia Américas ...",leche,13:43:23 22/08/23
2,leche lala entera 6 pzas 1 l c/u,142.0,Tlalpan,"Calzada De Tlalpan No. 1037, Colonia Américas ...",leche,13:43:23 22/08/23
3,leche lala 100 sin lactosa proteína 1 l,37.0,Tlalpan,"Calzada De Tlalpan No. 1037, Colonia Américas ...",leche,13:43:23 22/08/23
4,leche alpura clásica caja con 6 pzas de 1 l c/u,141.0,Tlalpan,"Calzada De Tlalpan No. 1037, Colonia Américas ...",leche,13:43:23 22/08/23
5,leche lala deslactosada 6 pzas de 1 l c/u,144.0,Tlalpan,"Calzada De Tlalpan No. 1037, Colonia Américas ...",leche,13:43:23 22/08/23
6,leche lala 100 sin lactosa reducida en grasa l...,35.5,Tlalpan,"Calzada De Tlalpan No. 1037, Colonia Américas ...",leche,13:43:23 22/08/23
7,leche santa clara deslactosada caja con 6 pzas...,150.0,Tlalpan,"Calzada De Tlalpan No. 1037, Colonia Américas ...",leche,13:43:23 22/08/23
8,leche alpura deslactosada caja con 6 pzas de 1...,141.0,Tlalpan,"Calzada De Tlalpan No. 1037, Colonia Américas ...",leche,13:43:23 22/08/23
9,leche fresca lala 100 sin lactosa proteína 1 l,37.0,Tlalpan,"Calzada De Tlalpan No. 1037, Colonia Américas ...",leche,13:43:23 22/08/23


In [43]:
url="https://super.walmart.com.mx/"
driver=webdriver.Edge(options=options)
driver.get(url)
time.sleep(5)

In [44]:
try:
    elemento = driver.find_element(By.NAME, 'select-store-button')
    
except NoSuchElementException:
    elemento = driver.find_element(By.CSS_SELECTOR, 'img[src="https://i5.walmartimages.com/dfw/4ff9c6c9-14f0/k2-_8ce34043-6050-4f39-afb2-3e76a554f8a3.v1.png"]')

elemento.click()

In [45]:
#Código postal
time.sleep(4)
cp=driver.find_element(By.CSS_SELECTOR,'input[data-automation-id="store-zip-code"]')
cp.clear()
cp.send_keys('58090')
time.sleep(4)
tienda = driver.find_element(By.CSS_SELECTOR, 'span.b.f5.lh-copy.dark-gray').text
address = driver.find_element(By.CSS_SELECTOR, 'span.f6[aria-label]').get_attribute('aria-label')
radio_buttons = driver.find_element(By.NAME,'pickup-store')
radio_buttons.click()
#Elegir
elegir=driver.find_element(By.CSS_SELECTOR,'button[data-automation-id="save-label"]')
elegir.click()

In [46]:
time.sleep(5)
url_busqueda="https://super.walmart.com.mx/search?q=atún+en+lata"
driver.get(url_busqueda)
time.sleep(5)
html=driver.page_source


In [47]:
soup = BeautifulSoup(html, 'html.parser')

In [48]:
results = soup.find_all('div', {'data-stack-index': '0'})

In [49]:
span_tags = []
product_titles = []

# Loop through each individual result and select the desired elements
for result in results:
    span_tags.extend(result.select('div[data-automation-id="product-price"] span'))
    product_titles.extend([title.text for title in result.select('span[data-automation-id="product-title"]')])

precios = []

for span in span_tags:
    text = span.text
    if 'precio actual' in text:
        precios.append(text.split("precio actual")[1].strip())

In [50]:
len(product_titles), len(precios)

(44, 44)

In [51]:
#Llevar product_title y precios a un dataframe
df=pd.DataFrame({'producto':product_titles,'precio':precios})
df

,producto,precio
0,Atún Great Value en agua con aceite 140 g,$13.50
1,Atún Great Value en agua 140 g,$13.50
2,Atún ahumado estilo marlín por kilo,$272.00/kg
3,Atún Dolores aleta amarilla en agua 140 g,$21.50
4,Atún Dolores aleta amarilla en agua 295 g,$40.50
5,Atún Tuny clásico en agua 140 g,$21.00
6,Atún Tuny light aleta amarilla en agua jumbo 2...,$39.50
7,Atún Dolores aleta amarilla en agua con aceite...,$21.50
8,Atún Dolores aleta amarilla en agua con aceite...,$40.50
9,Medallones de atún Marketside premium aleta am...,$269.00


In [52]:
tienda

'Altozano'

In [53]:
address

'Av. Montaña Monarca Norte No. 250, Colonia Ejido Jesús Del Monte, Morelia, Michoacan 58297.'